In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from src.models.load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

In [2]:
access_path = './data/processed/'
# bal_from_nm = 'bal_btw_rebal_20210602.dat'
# last_rebal_nm = 'balance_rebaldate_0702_all.dat'
last_rebal_nm = 'balance_rebaldate_0801_all.dat'

In [3]:
bal_from = pd.read_csv(access_path + last_rebal_nm)
price_db = PriceDB.instance().data

In [4]:
price_db

,date,itemcode,price,volume,trading_amt_mln,ret,itemtype
0,2018-07-20,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
1,2018-07-21,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
2,2018-07-22,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
3,2018-07-23,A069500,27685.0000,7283838.0,219684.583805,-0.008094,ETF
4,2018-07-24,A069500,27770.0000,2562621.0,170177.717593,0.003066,ETF
...,...,...,...,...,...,...,...
39526,2021-08-27,CALL,0.0086,NaN,NaN,NaN,riskfree
39527,2021-08-28,CALL,0.0086,NaN,NaN,NaN,riskfree
39528,2021-08-29,CALL,0.0086,NaN,NaN,NaN,riskfree
39529,2021-08-30,CALL,0.0086,NaN,NaN,NaN,riskfree


#### MP 발생일
- 2021.5.3
- 2021.6.2
- 2021.7.2
- 2021.8.1 (일요일)
- 2021.9.1

In [5]:
# 두 MP발생일 사이 기간 설정
start_dt = '20210802'  # MP발생 다음날부터
end_dt = '20210831'    # MP발생 전날까지

range_dt_ymd = pd.date_range(start=start_dt, end=end_dt, freq='D')  # price_db 날짜 포맷
range_dt_mdy = range_dt_ymd.strftime('%-m/%-d/%Y')  # 잔고(balance) 날짜 포맷

In [6]:
# price_db 마지막 날짜를 휴일에 복사하기
# temp = price_db[price_db.date==price_db.iloc[-1,0]].copy()
# temp.date = '2021-07-31'
# price_db = pd.concat((price_db, temp), axis=0)
# price_db = price_db.reset_index(drop=True)

#### 최근 과거 MP발생일 잔고를 해당 기간 내 매일 매일의 잔고로 복사해 두고, price 컬럼과 value 컬럼을 업데이트하면 끝.

- 잔고 복사

In [7]:
df_list = []
for cnt, dt in enumerate(range_dt_ymd):
    temp = bal_from.copy(deep=True)
    temp.date = dt
    df_list.append(temp)
    
bal_btw_rebal = pd.concat(df_list)

- `price_db`를 이용해서 Price 컬럼 업데이트.

In [8]:
bal_btw_rebal = bal_btw_rebal.set_index(['date', 'itemcode'])
price_db = price_db.set_index(['date', 'itemcode'])
bal_btw_rebal.update(price_db)
bal_btw_rebal = bal_btw_rebal.reset_index(drop=False)

- 사용자별/날짜별로 비중(`wt`) 계산

In [9]:
bal_btw_rebal.value = bal_btw_rebal.quantity*bal_btw_rebal.price
bal_btw_rebal.original = ''
bal_btw_rebal.wt = bal_btw_rebal.value/bal_btw_rebal.groupby(['date', 'userid'])['value'].transform('sum')

In [10]:
bal_btw_rebal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,2021-08-02,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2.0,58030.0,116060.0,58030.0,116060.0,0.162358,NaN,
1,2021-08-02,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,1.0,101240.0,101240.0,101230.0,101230.0,0.141612,NaN,
2,2021-08-02,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2.0,101140.0,202280.0,101145.0,202290.0,0.282986,NaN,
3,2021-08-02,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1.0,104495.0,104495.0,104500.0,104500.0,0.146187,NaN,
4,2021-08-02,A273130,A001,안정추구형소규모,Fixed Income,KODEX 종합채권(AA-이상)액티브,1.0,109690.0,109690.0,109695.0,109695.0,0.153454,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2021-08-31,A237370,A009,적극투자형대규모,Alternative,KODEX 배당성장채권혼합,27.0,12165.0,328455.0,12130.0,327510.0,0.245409,NaN,
2156,2021-08-31,A102780,A009,적극투자형대규모,Equity,KODEX 삼성그룹,17.0,10475.0,178075.0,10555.0,179435.0,0.134454,NaN,
2157,2021-08-31,A069500,A009,적극투자형대규모,Equity,KODEX 200,3.0,42485.0,127455.0,41985.0,125955.0,0.094380,NaN,
2158,2021-08-31,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,1.0,58030.0,58030.0,58140.0,58140.0,0.043565,NaN,


In [11]:
bal_btw_rebal.to_csv(access_path + 'bal_btw_rebal.dat', index=False, encoding='utf-8-sig')

In [12]:
bal_btw_rebal.columns

Index(['date', 'itemcode', 'userid', 'name', 'asset_class', 'itemname',
       'quantity', 'cost_price', 'cost_value', 'price', 'value', 'wt',
       'group_by', 'original'],
      dtype='object')